In [32]:
from langchain_community.document_loaders import DirectoryLoader,TextLoader, UnstructuredURLLoader, PyPDFLoader

# loader = DirectoryLoader('', glob="**/*.txt",loader_cls=TextLoader)
# urls = [
#     "https://www.alltius.ai/company/about",
# ]
loader = PyPDFLoader("Chapter 1.pmd.pdf")

text_documents = loader.load()
text_documents

[Document(page_content='1 COMPONENTS  OF FOOD\n1\nIn lower classes, we made lists of the\nfood items that we eat. We also\nidentified food items eaten in\ndifferent parts of India and marked\nthese on its map.\nA meal could consist of  chapati, dal\nand brinjal curry. Another may be rice,sambar and a vegetable preparation of\nlady’s finger  (bhindi). Yet another meal\ncould be appam , fish curry and\nvegetables.curd, butter milk and pickles. Some\nexamples of meals from different regions\nare given in Table 1.1. Select food itemsand enter these in Table 1.1.\nSometimes, we may not really have\nall this variety in our meals. If we aretravelling, we may eat whatever is\navailable on the way. It may not be\npossible for some of us, to eat such avariety of items, most of the time.\nThere must be some reason though,\nwhy meals usually consist of such adistribution. Do you think that our body\nneeds different kinds of food for some\nspecial purpose?\n1.1 W HAT DO DIFFERENT  FOOD\nITEMS CONTA

In [22]:
from langchain_community.document_loaders import PythonLoader
loader  = PythonLoader('../test.py')
text_documents = loader.load()

In [23]:
# from langchain_community.document_loaders import WebBaseLoader
# import bs4

# loader = WebBaseLoader(web_paths=['https://eshtyle.com/terms-conditions','https://eshtyle.com/privacy-policy'],
#                        bs_kwargs=dict(parse_only=bs4.SoupStrainer(
# )))
                    
# loader.load()

In [24]:
text_documents

[Document(page_content='import streamlit as st\nimport os\nfrom langchain_groq import ChatGroq\nfrom langchain_fireworks import FireworksEmbeddings\nfrom langchain.text_splitter import RecursiveCharacterTextSplitter\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom langchain.chains import create_retrieval_chain\nfrom langchain_community.vectorstores import FAISS\nfrom langchain_community.document_loaders import  PyPDFLoader\nfrom langchain_community.document_loaders import DirectoryLoader,TextLoader\nfrom dotenv import load_dotenv\n\nload_dotenv()\napi_key = os.environ.get("FIREWORKS_API_KEY")\n\n## load the GROQ And OpenAI API KEY \ngroq_api_key=os.getenv(\'GROQ_API\')\n\nst.title("Chat bot for ecom website eshtyle.com")\n\nllm=ChatGroq(groq_api_key=groq_api_key,\n             model_name="Llama3-8b-8192")\n\nprompt=ChatPromptTemplate.from_template(\n"""\nAnswer the questions based on the provided c

In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=10)
document = text_splitter.split_documents(text_documents)
document

[Document(page_content='1 COMPONENTS  OF FOOD\n1\nIn lower classes, we made lists of the\nfood items that we eat. We also\nidentified food items eaten in\ndifferent parts of India and marked\nthese on its map.\nA meal could consist of  chapati, dal\nand brinjal curry. Another may be rice,sambar and a vegetable preparation of\nlady’s finger  (bhindi). Yet another meal\ncould be appam , fish curry and\nvegetables.curd, butter milk and pickles. Some\nexamples of meals from different regions\nare given in Table 1.1. Select food itemsand enter these in Table 1.1.\nSometimes, we may not really have\nall this variety in our meals. If we aretravelling, we may eat whatever is\navailable on the way. It may not be\npossible for some of us, to eat such avariety of items, most of the time.\nThere must be some reason though,\nwhy meals usually consist of such adistribution. Do you think that our body\nneeds different kinds of food for some\nspecial purpose?\n1.1 W HAT DO DIFFERENT  FOOD\nITEMS CONTA

In [26]:
# from langchain.embeddings import HuggingFaceInstructEmbeddings
# instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", 
#                                                       model_kwargs={"device": "cpu"})

In [27]:
# from langchain_community.embeddings import HuggingFaceBgeEmbeddings


# embeddings = embeddings(
#     query_instruction="Represent the query for retrieval: "
# )

In [34]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.environ.get("FIREWORKS_API_KEY")

In [35]:
from langchain_fireworks import FireworksEmbeddings

instructor_embeddings = FireworksEmbeddings(model="nomic-ai/nomic-embed-text-v1.5")

In [36]:
from langchain_community.vectorstores import Chroma
vectorstore  = Chroma.from_documents(document,instructor_embeddings)

In [37]:
## FAISS Vector Database
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(document, instructor_embeddings)

In [38]:
retriever=db.as_retriever(search_type="similarity",search_kwargs={"k":3})


In [39]:
import os

In [40]:
groq_api_key=os.getenv('GROQ_API')


In [41]:
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=groq_api_key,
             model_name="Llama3-8b-8192")

In [42]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True)

In [44]:
from langchain.chains import (
     ConversationalRetrievalChain
)


In [45]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


In [46]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
"""
Answer the questions using provided content and use general knowledge when necessary.
Please provide the most accurate response based on the question
<context>
{context}
<context>
Questions:{input}

"""
)

In [48]:
document_chain=create_stuff_documents_chain(llm,prompt)
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [55]:
response=retrieval_chain.invoke({'input':"can you tell me fill in the blanks (a) question from pdf"})

In [56]:
response['answer']

'Here are the answers to the fill-in-the-blanks:\n\n(a) Carbohydrates are also called "energy giving foods".\n\nNote: The provided context is a chapter on nutrition and food testing, and the questions are related to the content.'

In [1]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

In [2]:
template = """You are a chatbot having a conversation with a human.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [8]:
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

c:\Users\91908\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [9]:
llm_chain.predict(human_input="Hi there my name is hrithik")





> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.


Human: Hi there my name is hrithik
Chatbot:

> Finished chain.


"Nice to meet you, Hrithik! It's great to have a conversation with you. How's your day going so far?"

In [10]:
llm_chain.predict(human_input="do you my name")





> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.

Human: Hi there my name is hrithik
AI: Nice to meet you, Hrithik! It's great to have a conversation with you. How's your day going so far?
Human: do you my name
Chatbot:

> Finished chain.


'Nice and easy! Yes, I remember, your name is Hrithik.'